In [102]:
import pandas as pd
import numpy as np
crsp = pd.read_pickle('Data/crsp.pkl')
bm = pd.read_csv('Data/Fundamentals/BM.csv')
cash_assets = pd.read_csv('Data/Fundamentals/Cash.csv')
gross_profitability = pd.read_csv('Data/Fundamentals/GP.csv')
mlev = pd.read_csv('Data/Fundamentals/Leverage.csv')
poy = pd.read_csv('Data/Fundamentals/PayoutYield.csv')
sales_growth = pd.read_csv('Data/Fundamentals/sgr.csv')
assets_growth = pd.read_csv('Data/Fundamentals/AssetGrowth.csv')
RoE = pd.read_csv('Data/Fundamentals/RoE.csv')
am = pd.read_csv('Data/Fundamentals/AM.csv')
blev = pd.read_csv('Data/Fundamentals/BookLeverage.csv')
asset_turnover = pd.read_csv('Data/Fundamentals/AssetTurnover.csv')


In [103]:
bm.rename(columns={'BM':'logBM'}, inplace=True)
bm.rename(columns={'yyyymm':'date'}, inplace=True)
cash_assets.rename(columns={'yyyymm':'date'}, inplace=True)
gross_profitability.rename(columns={'yyyymm':'date'}, inplace=True)
mlev.rename(columns={'yyyymm':'date'}, inplace=True)
poy.rename(columns={'yyyymm':'date'}, inplace=True)
sales_growth.rename(columns={'yyyymm':'date'}, inplace=True)
assets_growth.rename(columns={'yyyymm':'date'}, inplace=True)
RoE.rename(columns={'yyyymm':'date'}, inplace=True)
am.rename(columns={'yyyymm':'date'}, inplace=True)
blev.rename(columns={'yyyymm':'date'}, inplace=True)
asset_turnover.rename(columns={'yyyymm':'date'}, inplace=True)

In [104]:
import pandas as pd

def convert_yyyymm_to_period(series):
    """
    Converts a Series of dates in yyyymm format to Period[M] objects (i.e., yyyy-mm).
    Handles strings or integers, but not full date strings like yyyy-mm-dd.
    """
    series = series.astype(str).str.replace('-', '').str.zfill(6)

    # Ensure values are purely digits and 6 characters long
    mask = series.str.match(r'^\d{6}$')
    if not mask.all():
        raise ValueError("Some values in the series are not in yyyymm format.")

    return pd.to_datetime(series + '01', format='%Y%m%d')


bm['date'] = convert_yyyymm_to_period(bm['date'])
cash_assets['date'] = convert_yyyymm_to_period(cash_assets['date'])
gross_profitability['date'] = convert_yyyymm_to_period(gross_profitability['date'])
mlev['date'] = convert_yyyymm_to_period(mlev['date'])
poy['date'] = convert_yyyymm_to_period(poy['date'])
sales_growth['date'] = convert_yyyymm_to_period(sales_growth['date'])
assets_growth['date'] = convert_yyyymm_to_period(assets_growth['date'])
RoE['date'] = convert_yyyymm_to_period(RoE['date'])
am['date'] = convert_yyyymm_to_period(am['date'])
blev['date'] = convert_yyyymm_to_period(blev['date'])
asset_turnover['date'] = convert_yyyymm_to_period(asset_turnover['date'])


In [109]:
blev

,permno,date,BookLeverage
0,10000,1987-04-01,5.059808
1,10000,1987-05-01,5.059808
2,10000,1987-06-01,5.059808
3,10000,1987-07-01,5.059808
4,10000,1987-08-01,5.059808
...,...,...,...
3544850,93436,2025-01-01,1.700040
3544851,93436,2025-02-01,1.700040
3544852,93436,2025-03-01,1.700040
3544853,93436,2025-04-01,1.700040


In [213]:
from functools import reduce
dfs = [crsp, bm, cash_assets, gross_profitability, mlev, poy, sales_growth, assets_growth, RoE, am, blev, asset_turnover]

# Reduce merges all DataFrames on 'permno' and 'date'
df = reduce(lambda left, right: pd.merge(left, right, on=['permno', 'date'], how='inner'), dfs)

In [214]:
#Baseline Variables
df.rename(columns={'book_equity': 'BE'}, inplace=True)
df.rename(columns={'market_equity': 'ME'}, inplace=True)
df['payout'] = df['PayoutYield'] * df['ME'].shift(6)
df['at'] = df['AM'] * df['ME']

#Valuation Measures
df.rename(columns={'logBM': 'bm'}, inplace=True)
df['poy'] = np.log(1+ df['payout']/df['ME'])
df['sales'] = df['AssetTurnover'] * df['at']
df['yy'] = np.log(df['sales']/df['ME'])

#Growth Measures
df['beg'] = np.log(df['BE']/df['BE'].shift(1))
df['ag'] = np.log(df['AssetGrowth'])
df['yg'] = np.log(df['sgr'])

#Profitability Measures
df['csprof'] = np.log(1+ (df['payout']+df['BE'].diff())/df['BE'].shift(1))
df['roe'] = np.log(df['RoE'])
df['gprof'] = np.log(df['GP'])

#Capital Structure Measures
df.rename(columns={'Leverage': 'mlev'}, inplace=True)
df['total_debt'] = (df['mlev']*df['ME'])/(1-df['mlev'])
df['blev'] = np.log(df['BookLeverage'])
df.rename(columns={'Cash': 'cash'}, inplace=True)


In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import fsolve
import warnings
warnings.filterwarnings('ignore')

# =====================================
# STEP 1: DEFINE ALL VARIABLES
# =====================================

# You need to provide these as pandas DataFrames with:
# - Rows: firm-year observations
# - Columns: the variables defined below

# === Valuation Measures ===
# bm = log book-to-market ratio
# poy = payout yield = ln(1 + PO/ME)
# yy = sales yield = ln(Y/ME)

# === Growth Measures ===
# beg = book equity growth = ln(BE_t/BE_t-1)
# ag = asset growth = ln(A_t/A_t-1)
# yg = sales growth = ln(Y_t/Y_t-1)

# === Profitability Measures ===
# csprof = clean surplus profitability = ln(1 + (PO + ΔBE)/BE_t-1)
# roe = return on equity = ln(1 + E/((BE_t + BE_t-1)/2))
# gprof = gross profitability = ln(1 + GP/((A_t + A_t-1)/2))

# === Capital Structure Measures ===
# mlev = market leverage = B/(ME + B)
# blev = book leverage = B/A
# cash = cash holdings = C/A

# === Other Required Variables ===
# BE = book equity
# ME = market equity
# firm_id = firm identifier
# year = year

# Example structure (you'll replace with your actual data):
# df = pd.DataFrame({
#     'firm_id': [...],
#     'year': [...],
#     'BE': [...],
#     'ME': [...],
#     'bm': [...],
#     'poy': [...],
#     'yy': [...],
#     'beg': [...],
#     'ag': [...],
#     'yg': [...],
#     'csprof': [...],
#     'roe': [...],
#     'gprof': [...],
#     'mlev': [...],
#     'blev': [...],
#     'cash': [...]
# })



In [111]:
# Since the paper uses annual data, we need to aggregate monthly data
# We'll use June of each year (month 6) to match the paper's methodology

# Ensure date is datetime
df['date'] = pd.to_datetime(df['date'])

# Extract year and month
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month

# Filter to June observations only (or December if that's when fiscal year ends)
# You can adjust this based on your fiscal year convention
df_annual = df[df['month'] == 6].copy()

# Alternatively, if you want to use December values for market equity
# but annual accounting data, you might need a more complex merge
# For now, we'll use June to match the paper

# Sort by permno and year
df_annual = df_annual.sort_values(['permno', 'year'])

In [127]:
# =====================================
# STEP 2: SET PARAMETERS
# =====================================

# Number of periods for finite sum
H = 100

# Adjustment parameter for VAR stability
theta = 0.8  # Since theta^10 = 0.1

# State variable names in order
state_vars = ['bm', 'poy', 'yy', 'beg', 'ag', 'yg', 
              'csprof', 'roe', 'gprof', 'mlev', 'blev', 'cash']

# Number of state variables
n_vars = len(state_vars)



In [128]:
# =====================================
# STEP 3: PREPARE VAR DATA
# =====================================

# Assuming df is your DataFrame with all the required variables
# First, we need to create the state matrix for VAR estimation

# Create lagged variables
for var in state_vars:
    df[f'{var}_lag'] = df.groupby('permno')[var].shift(1)

# Remove observations with missing lags
df_var = df.dropna(subset=[f'{var}_lag' for var in state_vars])


In [165]:
# =====================================
# STEP 4: ESTIMATE VAR PARAMETERS (ROBUST VERSION)
# =====================================

# First, let's check and clean the data more thoroughly
print("Data cleaning and validation...")

# Remove any infinite values
for var in state_vars + [f'{var}_lag' for var in state_vars]:
    inf_count = np.isinf(df_var[var]).sum()
    if inf_count > 0:
        print(f"  Removing {inf_count} infinite values from {var}")
        df_var[var] = df_var[var].replace([np.inf, -np.inf], np.nan)

# Check for extreme values that might cause numerical issues
print("\nChecking for extreme values:")
for var in state_vars:
    q99 = df_var[var].quantile(0.99)
    q01 = df_var[var].quantile(0.01)
    extreme_count = ((df_var[var] > q99 * 10) | (df_var[var] < q01 * 10)).sum()
    if extreme_count > 0:
        print(f"  {var}: {extreme_count} extreme values detected")
        # Winsorize extreme values
        df_var[var] = df_var[var].clip(lower=df_var[var].quantile(0.001), 
                                       upper=df_var[var].quantile(0.999))

# Initialize storage for VAR parameters
Gamma = np.zeros((n_vars, n_vars))
intercepts = np.zeros(n_vars)

# Use more robust estimation approach
print("\nEstimating VAR parameters with robust methods...")

# Method 1: Try year-by-year first with careful handling
yearly_estimates = {i: {'intercepts': [], 'coefs': []} for i in range(n_vars)}

for year in sorted(df_var['year'].unique()):
    year_data = df_var[df_var['year'] == year]
    
    # Skip if too few observations
    if len(year_data) < 2 * n_vars:
        continue
    
    for i, dep_var in enumerate(state_vars):
        # Get data for this equation
        y = year_data[dep_var].values
        X = year_data[[f'{var}_lag' for var in state_vars]].values
        
        # Create valid data mask
        valid_mask = ~np.isnan(y) & ~np.any(np.isnan(X), axis=1)
        
        if np.sum(valid_mask) < n_vars + 5:
            continue
        
        y_valid = y[valid_mask]
        X_valid = X[valid_mask]
        
        # Standardize variables to improve numerical stability
        y_mean, y_std = np.mean(y_valid), np.std(y_valid)
        X_mean = np.mean(X_valid, axis=0)
        X_std = np.std(X_valid, axis=0)
        
        # Avoid division by zero
        y_std = y_std if y_std > 0 else 1
        X_std = np.where(X_std > 0, X_std, 1)
        
        # Standardize
        y_standard = (y_valid - y_mean) / y_std
        X_standard = (X_valid - X_mean) / X_std
        
        # Add intercept
        X_with_const = np.column_stack([np.ones(len(y_standard)), X_standard])
        
        try:
            # Use least squares with regularization
            from scipy.linalg import lstsq
            beta_standard, _, _, _ = lstsq(X_with_const, y_standard, 
                                          lapack_driver='gelsy')
            
            # Transform back to original scale
            beta = np.zeros(n_vars + 1)
            beta[0] = y_mean - np.sum(beta_standard[1:] * y_std * X_mean / X_std)
            beta[1:] = beta_standard[1:] * y_std / X_std
            
            yearly_estimates[i]['intercepts'].append(beta[0])
            yearly_estimates[i]['coefs'].append(beta[1:])
            
        except Exception as e:
            continue

# Aggregate yearly estimates
print("\nAggregating yearly estimates...")
for i in range(n_vars):
    if yearly_estimates[i]['coefs']:
        # Use median instead of mean for robustness
        intercepts[i] = np.median(yearly_estimates[i]['intercepts'])
        Gamma[i, :] = np.median(yearly_estimates[i]['coefs'], axis=0)
    else:
        print(f"  No valid estimates for equation {i} ({state_vars[i]})")

# Method 2: If year-by-year fails, use pooled estimation with regularization
if np.any(np.isnan(Gamma)) or np.all(Gamma == 0):
    print("\nUsing pooled estimation with regularization...")
    
    from sklearn.linear_model import Ridge
    
    for i, dep_var in enumerate(state_vars):
        # Get all data
        y = df_var[dep_var].values
        X = df_var[[f'{var}_lag' for var in state_vars]].values
        
        # Valid data
        valid_mask = ~np.isnan(y) & ~np.any(np.isnan(X), axis=1)
        
        if np.sum(valid_mask) > 100:  # Need reasonable amount of data
            y_valid = y[valid_mask]
            X_valid = X[valid_mask]
            
            # Ridge regression with cross-validation for regularization
            from sklearn.model_selection import cross_val_score
            
            alphas = [0.001, 0.01, 0.1, 1.0, 10.0]
            best_alpha = 0.1
            best_score = -np.inf
            
            for alpha in alphas:
                ridge = Ridge(alpha=alpha, fit_intercept=True)
                scores = cross_val_score(ridge, X_valid, y_valid, cv=5, 
                                       scoring='neg_mean_squared_error')
                if np.mean(scores) > best_score:
                    best_score = np.mean(scores)
                    best_alpha = alpha
            
            # Fit with best alpha
            ridge = Ridge(alpha=best_alpha, fit_intercept=True)
            ridge.fit(X_valid, y_valid)
            
            intercepts[i] = ridge.intercept_
            Gamma[i, :] = ridge.coef_
            print(f"  {dep_var}: Ridge regression successful (alpha={best_alpha})")

# Ensure stability of VAR
print("\nChecking and enforcing VAR stability...")

# Check eigenvalues
try:
    eigenvalues = np.linalg.eigvals(Gamma)
    max_eigenvalue = np.max(np.abs(eigenvalues))
    print(f"Initial max eigenvalue: {max_eigenvalue:.4f}")
    
    # If unstable, shrink towards zero
    if max_eigenvalue >= 0.99 or np.any(np.isnan(eigenvalues)):
        print("VAR is unstable or has numerical issues. Applying shrinkage...")
        
        # Shrink eigenvalues
        shrink_factor = 0.95 / max(max_eigenvalue, 1.0)
        Gamma = Gamma * shrink_factor
        
        # Recheck
        eigenvalues = np.linalg.eigvals(Gamma)
        max_eigenvalue = np.max(np.abs(eigenvalues))
        print(f"After shrinkage, max eigenvalue: {max_eigenvalue:.4f}")
        
except Exception as e:
    print(f"Eigenvalue calculation failed: {e}")
    print("Using conservative diagonal VAR as fallback...")
    
    # Fallback: Use diagonal VAR (each variable depends only on its own lag)
    Gamma = np.eye(n_vars) * 0.5  # Conservative persistence

# Final validation
print("\nFinal Gamma matrix validation:")
print(f"Contains NaN: {np.any(np.isnan(Gamma))}")
print(f"Contains Inf: {np.any(np.isinf(Gamma))}")
print(f"Max absolute value: {np.max(np.abs(Gamma)):.4f}")

# If still have issues, use very conservative values
if np.any(np.isnan(Gamma)) or np.any(np.isinf(Gamma)):
    print("\nUsing conservative default VAR parameters...")
    Gamma = np.eye(n_vars) * 0.3  # Low persistence
    intercepts = np.zeros(n_vars)  # Zero intercepts

print("\nGamma matrix (first 5x5):")
print(Gamma[:5, :5])
print(f"\nIntercepts (first 5): {intercepts[:5]}")

Data cleaning and validation...
  Removing 5 infinite values from yy
  Removing 10 infinite values from ag
  Removing 19 infinite values from yg
  Removing 4 infinite values from gprof
  Removing 10 infinite values from ag_lag
  Removing 19 infinite values from yg_lag
  Removing 1 infinite values from roe_lag

Checking for extreme values:
  poy: 17674 extreme values detected
  beg: 2 extreme values detected
  ag: 24 extreme values detected
  yg: 601 extreme values detected
  roe: 450343 extreme values detected
  mlev: 119626 extreme values detected
  blev: 339405 extreme values detected
  cash: 322 extreme values detected

Estimating VAR parameters with robust methods...

Aggregating yearly estimates...

Checking and enforcing VAR stability...
Initial max eigenvalue: 0.9965
VAR is unstable or has numerical issues. Applying shrinkage...
After shrinkage, max eigenvalue: 0.9467

Final Gamma matrix validation:
Contains NaN: False
Contains Inf: False
Max absolute value: 0.9966

Gamma matrix

In [171]:
# =====================================
# STEP 5: ESTIMATE COVARIANCE MATRIX (FIXED)
# =====================================

print("\nEstimating covariance matrix Sigma...")

# Method 1: Calculate residuals year by year and pool them
all_residuals = []
valid_residual_count = 0

for year in sorted(df_var['year'].unique()):
    year_data = df_var[df_var['year'] == year]
    
    if len(year_data) < 10:  # Skip years with too few observations
        continue
    
    # State matrix at t-1 (lagged values)
    X_lag = year_data[[f'{var}_lag' for var in state_vars]].values
    
    # State matrix at t (current values)
    X_current = year_data[state_vars].values
    
    # Check for valid data
    valid_mask = ~np.any(np.isnan(X_lag), axis=1) & ~np.any(np.isnan(X_current), axis=1)
    
    if np.sum(valid_mask) < 5:
        continue
    
    # Filter to valid observations
    X_lag_valid = X_lag[valid_mask]
    X_current_valid = X_current[valid_mask]
    
    # Predicted values: X_t = intercepts + X_t-1 * Gamma'
    X_pred = intercepts[np.newaxis, :] + X_lag_valid @ Gamma.T
    
    # Residuals
    residuals = X_current_valid - X_pred
    
    # Store residuals
    all_residuals.append(residuals)
    valid_residual_count += len(residuals)

print(f"Collected {valid_residual_count} valid residuals from {len(all_residuals)} years")

# Check if we have residuals
if all_residuals:
    # Stack all residuals
    all_residuals_stacked = np.vstack(all_residuals)
    print(f"Total residual matrix shape: {all_residuals_stacked.shape}")
    
    # Calculate covariance matrix
    # Remove any remaining NaN rows
    valid_rows = ~np.any(np.isnan(all_residuals_stacked), axis=1)
    clean_residuals = all_residuals_stacked[valid_rows]
    
    if len(clean_residuals) > n_vars:
        # Calculate covariance
        Sigma = np.cov(clean_residuals.T)
        print(f"Sigma calculated from {len(clean_residuals)} observations")
    else:
        print("Not enough valid residuals for covariance calculation")
        Sigma = np.eye(n_vars) * 0.1  # Default diagonal covariance
else:
    print("No valid residuals found!")
    Sigma = np.eye(n_vars) * 0.1  # Default diagonal covariance

# Validate Sigma
print("\nValidating Sigma matrix...")
print(f"Sigma shape: {Sigma.shape}")
print(f"Contains NaN: {np.any(np.isnan(Sigma))}")
print(f"Contains Inf: {np.any(np.isinf(Sigma))}")

# If Sigma has issues, try alternative estimation
if np.any(np.isnan(Sigma)) or np.any(np.isinf(Sigma)):
    print("\nSigma has numerical issues. Trying alternative estimation...")
    
    # Method 2: Direct calculation equation by equation
    Sigma = np.zeros((n_vars, n_vars))
    
    for i in range(n_vars):
        for j in range(i, n_vars):  # Only upper triangle due to symmetry
            cov_ij_values = []
            
            for year in sorted(df_var['year'].unique()):
                year_data = df_var[df_var['year'] == year]
                
                # Get residuals for variables i and j
                y_i = year_data[state_vars[i]].values
                y_j = year_data[state_vars[j]].values
                X_lag = year_data[[f'{var}_lag' for var in state_vars]].values
                
                # Valid observations
                valid_mask = ~np.isnan(y_i) & ~np.isnan(y_j) & ~np.any(np.isnan(X_lag), axis=1)
                
                if np.sum(valid_mask) < 5:
                    continue
                
                # Predictions
                pred_i = intercepts[i] + X_lag[valid_mask] @ Gamma[i, :]
                pred_j = intercepts[j] + X_lag[valid_mask] @ Gamma[j, :]
                
                # Residuals
                resid_i = y_i[valid_mask] - pred_i
                resid_j = y_j[valid_mask] - pred_j
                
                # Store covariance for this year
                cov_ij_values.append(np.mean(resid_i * resid_j))
            
            # Average across years
            if cov_ij_values:
                Sigma[i, j] = np.median(cov_ij_values)  # Use median for robustness
                Sigma[j, i] = Sigma[i, j]  # Symmetric
            else:
                # Default: small positive value on diagonal, zero off-diagonal
                Sigma[i, j] = 0.1 if i == j else 0.0
                Sigma[j, i] = Sigma[i, j]

# Ensure Sigma is positive definite
print("\nEnsuring Sigma is positive definite...")
try:
    # Check eigenvalues
    eigvals = np.linalg.eigvals(Sigma)
    min_eigval = np.min(eigvals)
    
    if min_eigval <= 0 or np.any(np.isnan(eigvals)):
        print(f"Sigma is not positive definite (min eigenvalue: {min_eigval})")
        
        # Fix using eigenvalue decomposition
        eigvals, eigvecs = np.linalg.eigh(Sigma)
        
        # Set minimum eigenvalue
        min_allowed = 0.001
        eigvals = np.maximum(eigvals, min_allowed)
        
        # Reconstruct
        Sigma = eigvecs @ np.diag(eigvals) @ eigvecs.T
        print("Fixed Sigma to be positive definite")
        
except Exception as e:
    print(f"Eigenvalue check failed: {e}")
    # Fallback: diagonal matrix
    Sigma = np.diag(np.ones(n_vars) * 0.1)

# Final check
print("\nFinal Sigma matrix check:")
print(f"Contains NaN: {np.any(np.isnan(Sigma))}")
print(f"Contains Inf: {np.any(np.isinf(Sigma))}")
print(f"Min eigenvalue: {np.min(np.linalg.eigvals(Sigma)):.6f}")
print(f"Max eigenvalue: {np.max(np.linalg.eigvals(Sigma)):.6f}")

# Display sample of Sigma
print("\nSigma matrix (first 5x5):")
print(Sigma[:5, :5])

# If still problematic, use very simple diagonal matrix
if np.any(np.isnan(Sigma)) or np.any(np.isinf(Sigma)):
    print("\nUsing simple diagonal Sigma as last resort...")
    Sigma = np.eye(n_vars) * 0.1


Estimating covariance matrix Sigma...
Collected 441304 valid residuals from 48 years
Total residual matrix shape: (441304, 12)
Sigma calculated from 441304 observations

Validating Sigma matrix...
Sigma shape: (12, 12)
Contains NaN: False
Contains Inf: False

Ensuring Sigma is positive definite...

Final Sigma matrix check:
Contains NaN: False
Contains Inf: False
Min eigenvalue: 0.000734
Max eigenvalue: 0.402323

Sigma matrix (first 5x5):
[[ 0.01990136  0.00040189  0.00161594 -0.01171618 -0.00575493]
 [ 0.00040189  0.01020704  0.00166511 -0.00193363 -0.00035582]
 [ 0.00161594  0.00166511  0.02775113 -0.01779469  0.00742662]
 [-0.01171618 -0.00193363 -0.01779469  0.07660605  0.00526991]
 [-0.00575493 -0.00035582  0.00742662  0.00526991  0.12132656]]


In [173]:
# =====================================
# STEP 6: ADJUST VAR FOR STABILITY
# =====================================

# Create steady-state Gamma (diagonal with zeros)
Gamma_ss = np.zeros_like(Gamma)

# Adjusted Gamma
Gamma_adj = theta * Gamma + (1 - theta) * Gamma_ss



In [175]:
# =====================================
# STEP 7: CALCULATE JENSEN'S INEQUALITY ADJUSTMENTS
# =====================================

# Define selector vectors
idx_csprof = state_vars.index('csprof')
idx_beg = state_vars.index('beg')

# Selector vectors
e_csprof = np.zeros(n_vars)
e_csprof[idx_csprof] = 1

e_beg = np.zeros(n_vars)
e_beg[idx_beg] = 1

e_po = e_csprof - e_beg  # for payout

# Calculate v1(h) and v2(h) for h = 1 to H
v1_values = np.zeros(H + 1)
v2_values = np.zeros(H + 1)

# For v1(h)
for h in range(1, H + 1):
    if h == 1:
        v1_values[h] = 0.5 * e_po @ Sigma @ e_po + e_po @ Sigma @ e_beg
    else:
        Gamma_power = np.linalg.matrix_power(Gamma_adj, h-1)
        v1_values[h] = v1_values[h-1] + 0.5 * e_po @ Gamma_power @ Sigma @ Gamma_power.T @ e_po
        
        # Calculate F1(h) = sum of Gamma^i from i=0 to h-1
        F1_h = np.eye(n_vars)
        for i in range(1, h):
            F1_h += np.linalg.matrix_power(Gamma_adj, i)
        
        v1_values[h] += e_po @ Gamma_power @ Sigma @ F1_h.T @ e_beg

# For v2(h) - more complex calculation
for h in range(1, H + 1):
    if h == 1:
        v2_values[h] = 0.5 * e_beg @ Sigma @ e_beg
    else:
        # Need to calculate covariance terms
        h_v2_h = (h-1) * v2_values[h-1]
        
        # Add CovBEg_h,h term
        h_v2_h += 0.5 * e_beg @ Sigma @ e_beg
        
        # Add sum of CovBEg_h-i,h terms
        for i in range(1, h):
            # Calculate F2 matrix for covariance
            F2 = np.zeros((n_vars, n_vars))
            for j in range(i):
                if h-i+j < h:
                    F2 += np.linalg.matrix_power(Gamma_adj, j) @ Sigma @ np.linalg.matrix_power(Gamma_adj, h-i+j).T
            
            h_v2_h += e_beg @ F2 @ e_beg
        
        v2_values[h] = h_v2_h / h



In [178]:
# =====================================
# STEP 8: CALCULATE STEADY-STATE VALUES
# =====================================
idx_bm = state_vars.index('bm')
# Steady state: s_bar = (I - Gamma)^(-1) * intercepts
I = np.eye(n_vars)
s_bar = np.linalg.inv(I - Gamma) @ intercepts

# Market-to-book ratio steady state
MB_bar = np.exp(-s_bar[idx_bm])  # Since bm = log(BE/ME), so ME/BE = exp(-bm)



In [180]:
# =====================================
# STEP 9: SOLVE FOR COMMON DISCOUNT RATE
# =====================================

from scipy.optimize import fsolve

def calculate_mb_given_dr(dr):
    """Calculate implied MB ratio given discount rate dr"""
    mb_sum = 0
    
    for h in range(1, H + 1):
        # Calculate expected payout ratio
        Gamma_h = np.linalg.matrix_power(Gamma, h)
        
        # Expected log ratios
        exp_csprof_minus_beg = (e_csprof - e_beg) @ Gamma_h @ s_bar + v1_values[h]
        
        # Sum of expected BEg from 1 to h
        sum_exp_beg = 0
        for tau in range(1, h + 1):
            Gamma_tau = np.linalg.matrix_power(Gamma, tau)
            sum_exp_beg += e_beg @ Gamma_tau @ s_bar
        
        sum_exp_beg += h * v2_values[h]
        
        # Expected payout ratio
        exp_po_ratio = (np.exp(exp_csprof_minus_beg) - 1) * np.exp(sum_exp_beg)
        
        # Discount and add to sum
        mb_sum += exp_po_ratio * np.exp(-h * dr)
    
    # Add terminal value
    h = H
    Gamma_h = np.linalg.matrix_power(Gamma, h)
    exp_csprof_minus_beg = (e_csprof - e_beg) @ Gamma_h @ s_bar + v1_values[h]
    sum_exp_beg = 0
    for tau in range(1, h + 1):
        Gamma_tau = np.linalg.matrix_power(Gamma, tau)
        sum_exp_beg += e_beg @ Gamma_tau @ s_bar
    sum_exp_beg += h * v2_values[h]
    
    exp_po_ratio_H = (np.exp(exp_csprof_minus_beg) - 1) * np.exp(sum_exp_beg)
    
    # Terminal value calculation
    beg_bar = e_beg @ s_bar
    terminal_growth = beg_bar + v2_values[H] - dr
    
    if terminal_growth < 0:  # Ensure convergence
        pv_cf_ratio = np.exp(terminal_growth) / (1 - np.exp(terminal_growth))
        mb_sum += exp_po_ratio_H * np.exp(-H * dr) * pv_cf_ratio
    
    return mb_sum

# Solve for dr such that calculated MB equals steady-state MB
dr_initial_guess = 0.1  # 10% initial guess

# Define the equation to solve
def mb_equation(dr):
    return calculate_mb_given_dr(dr) - MB_bar

# Solve for dr
dr_solution = fsolve(mb_equation, dr_initial_guess)[0]
print(f"Common discount rate dr = {dr_solution:.4f}")



Common discount rate dr = 0.3158


In [182]:
# Check data types of state variables
print("Data types of state variables:")
for var in state_vars:
    print(f"{var}: {df_annual[var].dtype}")

# Convert to numeric if needed
for var in state_vars:
    df_annual[var] = pd.to_numeric(df_annual[var], errors='coerce')

# Check for any remaining non-numeric values
print("\nMissing values after conversion:")
print(df_annual[state_vars].isna().sum())

Data types of state variables:
bm: float64
poy: float64
yy: float64
beg: float64
ag: float64
yg: float64
csprof: float64
roe: float64
gprof: float64
mlev: float64
blev: float64
cash: float64

Missing values after conversion:
bm            0
poy           0
yy            0
beg           0
ag        18044
yg        18293
csprof        0
roe       10952
gprof       791
mlev          0
blev          0
cash          0
dtype: int64


In [183]:
import numpy as np
import pandas as pd
from scipy.optimize import fsolve
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# =====================================
# STEP 10: CALCULATE FE FOR EACH FIRM (OPTIMIZED WITH PROGRESS BAR)
# =====================================

# First, check data quality
print("Checking data quality...")
print(f"Total observations: {len(df_annual)}")
print(f"Observations with complete state variables: {df_annual[state_vars].notna().all(axis=1).sum()}")

# Ensure all state variables are numeric
for var in state_vars:
    df_annual[var] = pd.to_numeric(df_annual[var], errors='coerce')

# Pre-compute matrix powers to avoid redundant calculations
print("\nPre-computing matrix powers (this speeds up calculation significantly)...")
H_practical = 100  # Reduce from 1000 to 100 for practical computation

# Pre-compute all Gamma powers
Gamma_powers = {}
Gamma_powers[0] = np.eye(n_vars)
Gamma_powers[1] = Gamma

for h in tqdm(range(2, H_practical + 1), desc="Computing Gamma powers"):
    Gamma_powers[h] = Gamma_powers[h-1] @ Gamma

# Pre-compute cumulative sums of Gamma powers for BEg calculation
print("\nPre-computing cumulative Gamma sums...")
Gamma_cumsum = {}
Gamma_cumsum[0] = np.zeros((n_vars, n_vars))

for h in tqdm(range(1, H_practical + 1), desc="Computing cumulative sums"):
    Gamma_cumsum[h] = Gamma_cumsum[h-1] + Gamma_powers[h]

# Vectorized FE calculation with batching
print(f"\nCalculating FE for {len(df_annual)} observations...")

# Function to calculate FE for a batch of firms
def calculate_fe_batch(batch_df, batch_idx, total_batches):
    """Calculate FE for a batch of firms efficiently"""
    batch_size = len(batch_df)
    fe_results = np.full(batch_size, np.nan)
    
    # Get state matrix for batch
    state_matrix = batch_df[state_vars].values
    be_values = batch_df['BE'].values
    
    # Find valid observations (no NaN in state variables and positive BE)
    valid_mask = (~np.any(np.isnan(state_matrix), axis=1)) & (be_values > 0)
    
    if not np.any(valid_mask):
        return fe_results
    
    # Process only valid observations
    valid_states = state_matrix[valid_mask]
    valid_be = be_values[valid_mask]
    n_valid = valid_states.shape[0]
    
    # Initialize FE/BE ratios
    fe_be_ratios = np.zeros(n_valid)
    
    # Calculate for each h (vectorized across firms)
    for h in range(1, H_practical + 1):
        # Vectorized calculation for all valid firms at once
        # exp_csprof_minus_beg for all firms
        exp_terms = valid_states @ Gamma_powers[h].T @ (e_csprof - e_beg) + v1_values[h]
        
        # Sum of expected BEg for all firms
        sum_beg_terms = valid_states @ Gamma_cumsum[h].T @ e_beg + h * v2_values[h]
        
        # Clip to prevent overflow
        exp_terms = np.clip(exp_terms, -100, 100)
        sum_beg_terms = np.clip(sum_beg_terms, -100, 100)
        
        # Expected payout ratios
        exp_po_ratios = (np.exp(exp_terms) - 1) * np.exp(sum_beg_terms)
        
        # Discount and add
        contributions = exp_po_ratios * np.exp(-h * dr_solution)
        fe_be_ratios += contributions
        
        # Early stopping if contributions become negligible
        if h > 20 and np.max(np.abs(contributions)) < 1e-10:
            break
    
    # Add simple terminal value
    if h == H_practical:
        # Use Gordon growth model approximation for terminal value
        terminal_growth = valid_states @ e_beg + v2_values[H_practical] - dr_solution
        
        # Only calculate terminal value where it converges
        converging_mask = terminal_growth < -0.01
        if np.any(converging_mask):
            terminal_pv_ratios = np.zeros_like(terminal_growth)
            terminal_pv_ratios[converging_mask] = (
                np.exp(terminal_growth[converging_mask]) / 
                (1 - np.exp(terminal_growth[converging_mask]))
            )
            
            # Last period's contribution times terminal PV ratio
            last_contributions = exp_po_ratios * np.exp(-H_practical * dr_solution)
            fe_be_ratios += last_contributions * terminal_pv_ratios
    
    # Calculate FE values
    fe_values = valid_be * fe_be_ratios
    
    # Place results back in correct positions
    fe_results[valid_mask] = fe_values
    
    return fe_results

# Process in batches with progress bar
batch_size = 1000  # Process 1000 firms at a time
n_batches = (len(df_annual) + batch_size - 1) // batch_size

all_fe_results = []

# Main progress bar for batches
with tqdm(total=len(df_annual), desc="Calculating FE") as pbar:
    for batch_idx in range(n_batches):
        # Get batch
        start_idx = batch_idx * batch_size
        end_idx = min((batch_idx + 1) * batch_size, len(df_annual))
        batch_df = df_annual.iloc[start_idx:end_idx]
        
        # Calculate FE for batch
        batch_results = calculate_fe_batch(batch_df, batch_idx, n_batches)
        all_fe_results.extend(batch_results)
        
        # Update progress bar
        pbar.update(len(batch_df))

# Add results to dataframe
df_annual['FE'] = all_fe_results

# Apply bounds as mentioned in the paper
print("\nApplying bounds to FE values...")
df_annual['FE_bounded'] = df_annual['FE'].copy()
lower_bound = df_annual[['ME', 'BE']].max(axis=1) / 100
upper_bound = df_annual[['ME', 'BE']].min(axis=1) * 100

# Only apply bounds where FE is not NaN
valid_fe_mask = df_annual['FE'].notna()
df_annual.loc[valid_fe_mask, 'FE_bounded'] = df_annual.loc[valid_fe_mask, 'FE'].clip(
    lower=lower_bound[valid_fe_mask], 
    upper=upper_bound[valid_fe_mask]
)

# Print summary statistics
print("\n" + "="*50)
print("FE CALCULATION COMPLETE")
print("="*50)
print(f"Total observations: {len(df_annual)}")
print(f"Valid FE values calculated: {df_annual['FE'].notna().sum()}")
print(f"Percentage with valid FE: {100 * df_annual['FE'].notna().sum() / len(df_annual):.1f}%")
print(f"\nFE statistics:")
print(df_annual['FE'].describe())
print(f"\nFE/ME ratio statistics:")
print((df_annual['FE'] / df_annual['ME']).describe())

# Optional: Save intermediate results to avoid re-computation
print("\nSaving results...")
df_annual.to_pickle('df_annual_with_fe.pkl')  # Save as pickle to preserve data types
# Or save as CSV: df_annual.to_csv('df_annual_with_fe.csv', index=False)
print("Results saved!")

Checking data quality...
Total observations: 70141
Observations with complete state variables: 41455

Pre-computing matrix powers (this speeds up calculation significantly)...


Computing Gamma powers: 100%|██████████| 99/99 [00:00<00:00, 199249.57it/s]



Pre-computing cumulative Gamma sums...


Computing cumulative sums: 100%|██████████| 100/100 [00:00<00:00, 658446.47it/s]



Calculating FE for 70141 observations...


Calculating FE: 100%|██████████| 70141/70141 [00:00<00:00, 247095.53it/s]



Applying bounds to FE values...

FE CALCULATION COMPLETE
Total observations: 70141
Valid FE values calculated: 41452
Percentage with valid FE: 59.1%

FE statistics:
count    4.145200e+04
mean    -8.953116e+07
std      3.680785e+09
min     -3.965668e+11
25%     -6.778961e+06
50%      4.343578e+06
75%      5.132976e+07
max      1.286701e+11
Name: FE, dtype: float64

FE/ME ratio statistics:
count    41452.000000
mean         0.273345
std          5.422839
min         -2.089470
25%         -0.024593
50%          0.039558
75%          0.126129
max        647.174446
dtype: float64

Saving results...
Results saved!


In [186]:
# =====================================
# STEP 10: CALCULATE FE FOR EACH FIRM (WITH PROGRESS BAR)
# =====================================

from tqdm import tqdm

# Now calculate FE for each firm-year observation
print(f"\nCalculating FE for {len(df_annual)} firm-year observations...")

fe_results = []

# Create progress bar
with tqdm(total=len(df_annual), desc="Calculating FE", unit="firms") as pbar:
    for idx, row in df_annual.iterrows():
        # Get current state vector
        try:
            # Try to convert to numeric values
            s_current = pd.to_numeric(row[state_vars], errors='coerce').values
            
            # Skip if any state variable is missing or non-numeric
            if np.any(np.isnan(s_current)):
                fe_results.append(np.nan)
                pbar.update(1)
                continue
                
        except Exception as e:
            # If conversion fails, append NaN and continue
            fe_results.append(np.nan)
            pbar.update(1)
            continue
        
        # Calculate FE/BE ratio
        fe_be_ratio = 0
        
        for h in range(1, H + 1):
            # Calculate expected payout ratio
            Gamma_h = np.linalg.matrix_power(Gamma, h)
            
            # Expected log ratios
            exp_csprof_minus_beg = (e_csprof - e_beg) @ Gamma_h @ s_current + v1_values[h]
            
            # Sum of expected BEg from 1 to h
            sum_exp_beg = 0
            for tau in range(1, h + 1):
                Gamma_tau = np.linalg.matrix_power(Gamma, tau)
                sum_exp_beg += e_beg @ Gamma_tau @ s_current
            
            sum_exp_beg += h * v2_values[h]
            
            # Expected payout ratio
            exp_po_ratio = (np.exp(exp_csprof_minus_beg) - 1) * np.exp(sum_exp_beg)
            
            # Discount and add to sum
            fe_be_ratio += exp_po_ratio * np.exp(-h * dr_solution)
        
        # Add terminal value
        h = H
        Gamma_h = np.linalg.matrix_power(Gamma, h)
        exp_csprof_minus_beg = (e_csprof - e_beg) @ Gamma_h @ s_current + v1_values[h]
        sum_exp_beg = 0
        for tau in range(1, h + 1):
            Gamma_tau = np.linalg.matrix_power(Gamma, tau)
            sum_exp_beg += e_beg @ Gamma_tau @ s_current
        sum_exp_beg += h * v2_values[h]
        
        exp_po_ratio_H = (np.exp(exp_csprof_minus_beg) - 1) * np.exp(sum_exp_beg)
        
        # Terminal value
        beg_current = e_beg @ s_current
        terminal_growth = beg_current + v2_values[H] - dr_solution
        
        if terminal_growth < 0:
            pv_cf_ratio = np.exp(terminal_growth) / (1 - np.exp(terminal_growth))
            fe_be_ratio += exp_po_ratio_H * np.exp(-H * dr_solution) * pv_cf_ratio
        
        # Calculate FE
        FE = row['BE'] * fe_be_ratio
        fe_results.append(FE)
        
        # Update progress bar
        pbar.update(1)
        
# Add FE to dataframe
df_annual['FE'] = fe_results

# Print summary
print(f"\nFE calculation complete!")
print(f"Valid FE values: {df_annual['FE'].notna().sum()} out of {len(df_annual)}")
print(f"Success rate: {100 * df_annual['FE'].notna().sum() / len(df_annual):.1f}%")


Calculating FE for 70141 firm-year observations...


Calculating FE: 100%|██████████| 70141/70141 [48:15<00:00, 24.23firms/s]    


FE calculation complete!
Valid FE values: 41452 out of 70141
Success rate: 59.1%


In [217]:
# =====================================
# STEP 11: CALCULATE KEY RATIOS
# =====================================

# Calculate fm = log(FE/ME)
df_annual['fm'] = np.log(df_annual['FE'] / df_annual['ME'])

# Calculate bf = log(BE/FE)
df_annual['bf'] = np.log(df_annual['BE'] / df_annual['FE'])

# Apply bounds to handle outliers (as mentioned in the paper)
# Bound FE at (1/100) * max(ME, BE) from below and 100 * min(ME, BE) from above
df_annual['FE_bounded'] = df_annual['FE'].copy()
lower_bound = df_annual[['ME', 'BE']].max(axis=1) / 100
upper_bound = df_annual[['ME', 'BE']].min(axis=1) * 100
df_annual['FE_bounded'] = df_annual['FE_bounded'].clip(lower=lower_bound, upper=upper_bound)

# Recalculate ratios with bounded FE
df_annual['fm_bounded'] = np.log(df_annual['FE_bounded'] / df_annual['ME'])
df_annual['bf_bounded'] = np.log(df_annual['BE'] / df_annual['FE_bounded'])

# Print summary statistics
print("\nSummary Statistics:")
print(f"Number of firms with FE calculated: {df_annual['FE'].notna().sum()}")
print(f"\nFE/ME ratio statistics:")
print(df_annual['fm_bounded'].describe())
print(f"\nBE/FE ratio statistics:")
print(df_annual['bf_bounded'].describe())

# Verify the identity: bm = fm + bf
df_annual['bm_check'] = df_annual['fm_bounded'] + df_annual['bf_bounded']
print(f"\nVerification that bm = fm + bf:")
print(f"Mean absolute difference: {np.abs(df_annual['bm'] - df_annual['bm_check']).mean():.6f}")

# =====================================
# STEP 12: MERGE BACK TO MONTHLY DATA (OPTIONAL)
# =====================================

# If you want to have FE values for all months in the original df
if 'FE' not in df.columns:
    print("\nMerging annual FE calculations back to monthly data...")
    
    # Select columns to merge
    merge_cols = ['permno', 'date', 'FE', 'FE_bounded', 'fm', 'bf', 'fm_bounded', 'bf_bounded']
    
    # Merge
    df = df.merge(
        df_annual[merge_cols], 
        on=['permno', 'date'], 
        how='left'
    )
    
    print(f"Merged FE data to {df['FE'].notna().sum()} monthly observations")

# Show final data structure
print("\nFinal data structure:")
print(f"df_annual shape: {df_annual.shape} (annual data with FE)")
print(f"df shape: {df.shape} (monthly data with FE merged)")

# Save results
print("\nSaving results...")
df_annual.to_csv('fundamental_equity_annual.csv', index=False)
# Or as pickle to preserve data types:
# df_annual.to_pickle('fundamental_equity_annual.pkl')

print("Analysis complete!")


Summary Statistics:
Number of firms with FE calculated: 41452

FE/ME ratio statistics:
count    41452.000000
mean        -2.969663
std          1.390754
min         -5.658329
25%         -4.162178
50%         -3.194198
75%         -2.073284
max          4.605170
Name: fm_bounded, dtype: float64

BE/FE ratio statistics:
count    41452.000000
mean         2.424840
std          2.107367
min         -4.605170
25%          1.022609
50%          2.664414
75%          4.605170
max          4.605170
Name: bf_bounded, dtype: float64

Verification that bm = fm + bf:
Mean absolute difference: 1.562950

Merging annual FE calculations back to monthly data...
Merged FE data to 41452 monthly observations

Final data structure:
df_annual shape: (70141, 106) (annual data with FE)
df shape: (770998, 103) (monthly data with FE merged)

Saving results...
Analysis complete!


In [193]:
fundamental_equity = df_annual.pivot(index='date', columns='permno', values='FE')
fm = df.pivot(index='date', columns='permno', values='fm')

In [219]:
pd.to_pickle(df_annual, 'df_annual.pkl')

In [199]:
price = pd.read_pickle('Data/price.pkl')
bookvalue = pd.read_pickle('Data/bookvalue.pkl')
dividends = pd.read_pickle('Data/dividends.pkl')
shrout = pd.read_pickle('Data/shrout.pkl')

In [208]:
# VOLLSTÄNDIGE RENDITEBERECHNUNG ÜBER PREIS-DIVIDENDEN-FORMEL
# Formel: R_t = (P_t + D_t) / P_{t-1} - 1

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Initialisierung der Listen für Equal Weight Berechnung
monthly_returns_long_ew = []
monthly_returns_short_ew = []
long_short_returns_clean = []

# Tracking für Portfolios über die Zeit
portfolio_tracking = []

for year in sorted(set(date.year for date in fm.index)):
    
    date = pd.Timestamp(f"{year}-06")
    date_dec = pd.Timestamp(f"{year-1}-12")
    
    if date_dec not in fm.index:
        continue
    if year == 2018:  # Daten enden 2018
        continue
    
    # Zeitrahmen: Juli des aktuellen Jahres bis Juni des folgenden Jahres
    month_range = pd.date_range(start=date + pd.DateOffset(months=1), periods=12, freq="ME")
    
    print(f"Processing year {year}...")
    
    # STOCK SELECTION (gleich wie vorher)
    clean_row = fm.loc[date_dec].dropna().sort_values(ascending=True)
    n = len(clean_row)
    if n == 0:
        continue
        
    k = int(n * 0.3)
    low_bm_permnos = clean_row.iloc[:k].index.tolist()   # Niedrige BM (Short)
    high_bm_permnos = clean_row.iloc[-k:].index.tolist() # Hohe BM (Long)
    
    # Speichere Portfolio-Zusammensetzung
    portfolio_tracking.append({
        'year': year,
        'rebalance_date': date,
        'high_bm_stocks': len(high_bm_permnos),
        'low_bm_stocks': len(low_bm_permnos)
    })
    
    # MONATLICHE RENDITEBERECHNUNG über Preis-Dividenden-Formel
    for i, current_month in enumerate(month_range):
        
        current_date = current_month
        
        # Bestimme vorherigen Monat
        if i == 0:
            # Erster Monat: verwende Rebalancing-Datum als Basis
            previous_date = date
        else:
            previous_date = month_range[i-1]
        
        print(f"  Calculating returns for {current_date.strftime('%Y-%m')}...")
        
        # === LONG PORTFOLIO (High BM) ===
        individual_returns_long = []
        
        for permno in high_bm_permnos:
            try:
                # Aktuelle Preise und Dividenden
                if current_date in price.index and permno in price.columns:
                    P_t = price.loc[current_date, permno]
                else:
                    continue
                    
                if previous_date in price.index and permno in price.columns:
                    P_t_minus_1 = price.loc[previous_date, permno]
                else:
                    continue
                
                # Dividenden für den aktuellen Monat
                if current_date in dividends.index and permno in dividends.columns:
                    D_t = dividends.loc[current_date, permno]
                    if pd.isna(D_t):
                        D_t = 0
                else:
                    D_t = 0
                
                # Prüfe auf gültige Werte
                if pd.isna(P_t) or pd.isna(P_t_minus_1) or P_t_minus_1 <= 0:
                    continue
                
                # RENDITEBERECHNUNG: R = (P_t + D_t) / P_{t-1} - 1
                stock_return = (P_t + D_t) / P_t_minus_1 - 1
                
                # Sanity Check: Extreme Renditen ausschließen
                if -0.95 <= stock_return <= 5.0:  # Zwischen -95% und +500%
                    individual_returns_long.append(stock_return)
                    
            except Exception as e:
                continue
        
        # === SHORT PORTFOLIO (Low BM) ===
        individual_returns_short = []
        
        for permno in low_bm_permnos:
            try:
                # Aktuelle Preise und Dividenden
                if current_date in price.index and permno in price.columns:
                    P_t = price.loc[current_date, permno]
                else:
                    continue
                    
                if previous_date in price.index and permno in price.columns:
                    P_t_minus_1 = price.loc[previous_date, permno]
                else:
                    continue
                
                # Dividenden für den aktuellen Monat
                if current_date in dividends.index and permno in dividends.columns:
                    D_t = dividends.loc[current_date, permno]
                    if pd.isna(D_t):
                        D_t = 0
                else:
                    D_t = 0
                
                # Prüfe auf gültige Werte
                if pd.isna(P_t) or pd.isna(P_t_minus_1) or P_t_minus_1 <= 0:
                    continue
                
                # RENDITEBERECHNUNG: R = (P_t + D_t) / P_{t-1} - 1
                stock_return = (P_t + D_t) / P_t_minus_1 - 1
                
                # Sanity Check: Extreme Renditen ausschließen
                if -0.95 <= stock_return <= 5.0:  # Zwischen -95% und +500%
                    individual_returns_short.append(stock_return)
                    
            except Exception as e:
                continue
        
        # EQUAL WEIGHT Portfolio-Renditen berechnen
        if len(individual_returns_long) > 0:
            portfolio_return_long = np.mean(individual_returns_long)
            monthly_returns_long_ew.append({
                'date': current_date,
                'return': portfolio_return_long,
                'n_stocks': len(individual_returns_long)
            })
        else:
            portfolio_return_long = np.nan
            
        if len(individual_returns_short) > 0:
            portfolio_return_short = np.mean(individual_returns_short)
            monthly_returns_short_ew.append({
                'date': current_date,
                'return': portfolio_return_short,
                'n_stocks': len(individual_returns_short)
            })
        else:
            portfolio_return_short = np.nan
        
        # LONG-SHORT SPREAD berechnen
        if not pd.isna(portfolio_return_long) and not pd.isna(portfolio_return_short):
            long_short_spread = portfolio_return_long - portfolio_return_short
            long_short_returns_clean.append({
                'date': current_date,
                'long_return': portfolio_return_long,
                'short_return': portfolio_return_short,
                'long_short_return': long_short_spread,
                'long_stocks': len(individual_returns_long),
                'short_stocks': len(individual_returns_short)
            })
            
            print(f"    Long: {portfolio_return_long:.4f} ({len(individual_returns_long)} stocks)")
            print(f"    Short: {portfolio_return_short:.4f} ({len(individual_returns_short)} stocks)")
            print(f"    Spread: {long_short_spread:.4f}")

# ERGEBNISSE ZUSAMMENFASSEN
print("\n=== CREATING FINAL DATAFRAMES ===")

# Long-Short Returns DataFrame
df_ls_clean = pd.DataFrame(long_short_returns_clean)
if not df_ls_clean.empty:
    df_ls_clean.set_index('date', inplace=True)
    df_ls_clean['cumulative_return'] = (1 + df_ls_clean['long_short_return']).cumprod() - 1

# Portfolio Tracking
df_portfolio = pd.DataFrame(portfolio_tracking)

# STATISTIKEN und PLOTS
if not df_ls_clean.empty:
    print("\n=== FINAL STATISTICS ===")
    print(f"Zeitraum: {df_ls_clean.index.min()} bis {df_ls_clean.index.max()}")
    print(f"Anzahl Monate: {len(df_ls_clean)}")
    print(f"Durchschnittliche monatliche Rendite: {df_ls_clean['long_short_return'].mean():.4f}")
    print(f"Annualisierte Rendite: {df_ls_clean['long_short_return'].mean() * 12:.4f}")
    print(f"Volatilität (monatlich): {df_ls_clean['long_short_return'].std():.4f}")
    print(f"Volatilität (annualisiert): {df_ls_clean['long_short_return'].std() * np.sqrt(12):.4f}")
    print(f"Sharpe Ratio: {(df_ls_clean['long_short_return'].mean() * 12) / (df_ls_clean['long_short_return'].std() * np.sqrt(12)):.4f}")
    print(f"Gesamtrendite: {df_ls_clean['cumulative_return'].iloc[-1]:.4f}")
    print(f"Durchschnittliche Anzahl Long-Aktien: {df_ls_clean['long_stocks'].mean():.1f}")
    print(f"Durchschnittliche Anzahl Short-Aktien: {df_ls_clean['short_stocks'].mean():.1f}")
    
    # PLOTS
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))
    
    # 1. Kumulative Rendite
    df_ls_clean['cumulative_return'].plot(ax=ax1, title="Kumulative Long-Short Rendite (Preis-Dividenden-Formel)")
    ax1.set_ylabel("Kumulative Rendite")
    ax1.grid(True)
    
    # 2. Monatliche Renditen
    df_ls_clean['long_short_return'].plot(ax=ax2, title="Monatliche Long-Short Renditen")
    ax2.set_ylabel("Monatliche Rendite")
    ax2.grid(True)
    
    # 3. Long vs Short Performance
    df_ls_clean[['long_return', 'short_return']].plot(ax=ax3, title="Long vs Short Portfolio Returns")
    ax3.set_ylabel("Monatliche Rendite")
    ax3.legend(['Long Portfolio', 'Short Portfolio'])
    ax3.grid(True)
    
    # 4. Anzahl Aktien im Portfolio
    df_ls_clean[['long_stocks', 'short_stocks']].plot(ax=ax4, title="Anzahl Aktien im Portfolio")
    ax4.set_ylabel("Anzahl Aktien")
    ax4.legend(['Long Portfolio', 'Short Portfolio'])
    ax4.grid(True)
    
    plt.tight_layout()
    plt.show()
    
    # Zusätzliche Diagnose
    print("\n=== DIAGNOSE ===")
    print("Letzte 10 Renditen:")
    print(df_ls_clean[['long_return', 'short_return', 'long_short_return']].tail(10))
    
    extreme_returns = df_ls_clean[(df_ls_clean['long_short_return'].abs() > 0.2)]
    print(f"\nAnzahl extremer Renditen (>20%): {len(extreme_returns)}")
    if len(extreme_returns) > 0:
        print("Extreme Renditen:")
        print(extreme_returns[['long_return', 'short_return', 'long_short_return']])

else:
    print("FEHLER: Keine gültigen Daten gefunden!")

# Speichere Ergebnisse für weitere Analyse
d#f_ls_clean.to_csv('long_short_returns_clean.csv')
print(f"\nErgebnisse gespeichert in 'long_short_returns_clean.csv'")


=== CREATING FINAL DATAFRAMES ===
FEHLER: Keine gültigen Daten gefunden!

Ergebnisse gespeichert in 'long_short_returns_clean.csv'


In [206]:
price_long_ts

[]